In [ ]:
pip install geopandas rasterio numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 51.3 MB/s eta 0:00:00


In [ ]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
import numpy as np

# File paths
mineralized_path = "/content/Mineralisation_points_buffer_500m.shp"
non_mineralized_path = "/content/Non_Mineralisation_points_buffer_500m.shp"
reference_raster_path = "/content/ferric_clip_normalized_30m.tif"  # Reference raster for resolution and CRS
output_label_tif = "cnn_labels_merged.tif"

# Load reference raster to get transform, shape, and CRS
with rasterio.open(reference_raster_path) as ref:
    raster_crs = ref.crs
    raster_transform = ref.transform
    raster_shape = (ref.height, ref.width)
    profile = ref.profile

# Read shapefiles and reproject to target CRS (EPSG:32643)
min_gdf = gpd.read_file(mineralized_path).to_crs(raster_crs)
non_gdf = gpd.read_file(non_mineralized_path).to_crs(raster_crs)

# Prepare shapes with corresponding labels
mineralized_shapes = [(geom, 1.0) for geom in min_gdf.geometry]
non_mineralized_shapes = [(geom, 0.0) for geom in non_gdf.geometry]

# Combine shapes from both shapefiles into a single list
combined_shapes = mineralized_shapes + non_mineralized_shapes

# Rasterize combined shapes into a label array
label_array = rasterize(
    shapes=combined_shapes,
    out_shape=raster_shape,
    transform=raster_transform,
    fill=np.nan,  # NaN for NoData value
    dtype='float32'
)

# Update raster profile for output
profile.update({
    "dtype": "float32",
    "count": 1,
    "nodata": np.nan,  # NaN as NoData value
    "compress": "lzw"
})

# Save the label raster as GeoTIFF
with rasterio.open(output_label_tif, "w", **profile) as dst:
    dst.write(label_array, 1)

print(f"✅ Saved merged label raster to: {output_label_tif}")


✅ Saved merged label raster to: cnn_labels_merged.tif
